In [1]:
from mtg_draft_ai.api import *

cards = read_cube_toml('cube_81183_tag_data.toml')

In [2]:
from mtg_draft_ai import synergy

G = synergy.create_graph(cards)

In [3]:
def contains_all_colors(cards, colors):
    colors_seen = set()
    for card in cards:
        if card.color_id != 'C':
            for color in card.color_id:
                colors_seen.add(color)
    
    return colors_seen == set(colors)

In [7]:
import networkx as nx
from networkx.algorithms import community
from mtg_draft_ai.display import cards_to_html

output_html = """
<style>
card-image {
    display: inline;
    margin: 1px;
}
</style>
"""

for colors in ['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG']:
    output_html += 'Colors: {} <br>\n'.format(colors)
    subgraph = synergy.colors_subgraph(G, colors)
    colors_gmc = community.greedy_modularity_communities(subgraph)

    i = 0
    for gmc in colors_gmc:
        if not contains_all_colors(gmc, colors):
            continue

        gmc_subgraph = subgraph.subgraph(gmc)
        sorted_community_cards = [k for k, v in synergy.sorted_centralities(gmc_subgraph)]
        
        output_html += 'Community {}\n'.format(i)
        output_html += '<div>\n{}</div>\n'.format(cards_to_html(sorted_community_cards))
        i += 1

print(output_html)



<style>
card-image {
    display: inline;
    margin: 1px;
}
</style>
Colors: WU <br>
Community 0
<div>
<img src="https://api.scryfall.com/cards/named?format=image&exact=Rise+from+the+Tides" width="146" height="204" class="card-image" />
<img src="https://api.scryfall.com/cards/named?format=image&exact=Naru+Meha%2C+Master+Wizard" width="146" height="204" class="card-image" />
<img src="https://api.scryfall.com/cards/named?format=image&exact=Thing+in+the+Ice" width="146" height="204" class="card-image" />
<img src="https://api.scryfall.com/cards/named?format=image&exact=Soulfire+Grand+Master" width="146" height="204" class="card-image" />
<img src="https://api.scryfall.com/cards/named?format=image&exact=Riddleform" width="146" height="204" class="card-image" />
<img src="https://api.scryfall.com/cards/named?format=image&exact=Jhessian+Thief" width="146" height="204" class="card-image" />
<img src="https://api.scryfall.com/cards/named?format=image&exact=Cryptic+Serpent" width="146" heig

In [8]:
with open('colorpairs.html', 'w') as f:
    f.write(output_html)

In [7]:
color_trios = ['WUB', 'UBR', 'BRG', 'RGW', 'GWU', 'WBR', 'URG', 'BGW', 'RWU', 'GUB']